### Imports

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib as mpl
import matplotlib.pyplot as plt
root = "/Users/phdenzel/gleam"
sys.path.append(root)
import gleam
from gleam.utils.lensing import LensModel
from gleam.utils.plotting import kappa_map_plot, plot_scalebar
import gleam.utils.colors as gcl
gcl.GLEAMcmaps.register_all()


### Load data

In [ ]:
#        ACS          ACS(WFC3)        ACS(WFC3)      WFC3(UNK)
#        ACS(WFC3)    SBC              WFC3           ACS(GAIA)
objs = ['B1608+656', 'DESJ0408-5354', 'HE0435-1223', 'PG1115+080',
        'RXJ0911+0551', 'RXJ1131-1231', 'SDSSJ1004+4112', 'WFIJ2033-4723']
objidx = 0
lens = objs[objidx]
print(lens)


In [ ]:
statedir = 'states/'
statefiles = ['11doubles_dg45.state',
              '11doubles_dg60.state', '11doubles_CMB_dg60.state', '11doubles_SNeIa_dg60.state',
              '7quads_dg45.state', '7quads_CMB_dg45.state', '7quads_SNeIa_dg45.state',
              '7quads_dg60.state', '7quads_CMB_dg60.state', '7quads_SNeIa_dg60.state', 
              'all_dg60.state', 'all_SNeIa_dg60.state']
statefile = statefiles[7]
print(statefile)


In [ ]:
lm = LensModel(statedir+statefile)
lm.obj_idx = objidx
print(lm.__v__)


In [ ]:
%%script false
# Prepare PCA data (refined grid)
dims = (2*lm.pixrad+1)*lm.env.objects[0].basis.subdivision
ea = lm.ensemble_average.reshape(dims**2)
darr = lm.data  # lm.data_toplevel
darr = darr.reshape((darr.shape[0], dims**2))
darr = darr - ea

print darr.shape, dims


In [ ]:
# Prepare PCA data (coarse grid)
dims = (2*lm.pixrad+1) # * lm.env.objects[0].basis.subdivision
ea = lm.kappa_grid(-1, refined=False)
ea_flat = ea.copy().reshape(dims**2)
darr = lm.data_toplevel  # lm.data
darr = darr.reshape((darr.shape[0], dims**2))
# darr = darr - ea_flat

print darr.shape, np.mean(darr)


### Run PCA

In [ ]:
# Standardize data
scaler = StandardScaler(with_std=False)  # calculate (data - mu) / sigma=1
scaler.fit(darr)
std_arr = scaler.transform(darr)


In [ ]:
Npc = darr.shape[1]
pca = PCA(n_components=Npc)
pca.fit(std_arr)
print std_arr.shape


In [ ]:
plt.plot(pca.explained_variance_, color=gcl.red)
plt.xlabel('npc')
plt.ylabel('eigval')
plt.tight_layout()
plt.savefig("results/{}_dist_eigval.pdf".format(lens))
plt.show()

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_), color=gcl.purpleblue)
plt.xlabel(r'$\Sigma$ npc')
plt.ylabel('cumul. variance ratio')
plt.tight_layout()
plt.savefig("results/{}_cumul_var_ratio.pdf".format(lens))
plt.show()

In [ ]:
pcacs = pca.components_
pcacs = pcacs.reshape((pcacs.shape[0], dims, dims))
print pcacs.shape


In [ ]:
for i in range(10):
    plt.imshow(pcacs[i], cmap='twilight')
    plt.axis('off')
    plt.gcf().axes[0].get_xaxis().set_visible(False)
    plt.gcf().axes[0].get_yaxis().set_visible(False)
    plt.colorbar()
    plt.show()
    plt.close()


In [ ]:
kappa_map_plot(lm, mdl_index=-1, obj_index=objidx, extent=lm.extent,
               contours=True, levels=7, delta=0.1, label=lm.obj_name)
plt.show()


In [ ]:
# calculate dot products between data vectors and the eigenvectors
npca = 0
warr = std_arr.reshape((1000, 17, 17)) * pcacs[npca, :]
dotprods = np.sum(warr, axis=(1, 2))
order_dotprods = dotprods.argsort()[::-1]
print dotprods.shape
print dotprods.max(), dotprods.min()
print dotprods[order_dotprods[160]], dotprods[order_dotprods[840]]
print order_dotprods[0:10], order_dotprods[-10:]

projections = np.array([d*pcacs[npca, :] for d in dotprods])
print projections.shape


In [ ]:
kappa_map_plot(lm, shift=projections[order_dotprods[160]], mdl_index=-1, obj_index=objidx, extent=lm.extent,
               contours=True, levels=7, delta=0.1, label=lm.obj_name)

In [ ]:
kappa_map_plot(lm, shift=projections[order_dotprods[840]], mdl_index=-1, obj_index=objidx, extent=lm.extent,
               contours=True, levels=7, delta=0.1, label=lm.obj_name)

In [ ]:
for npca in [0, 1, 2, 3, 4, 9, 24, 49, 99]:
    warr = std_arr.reshape((1000, 17, 17)) * pcacs[npca, :]
    dotprods = np.sum(warr, axis=(1, 2))
    order_dotprods = dotprods.argsort()[::-1]
    projections = np.array([d*pcacs[npca, :] for d in dotprods])
    kappa_map_plot(lm, shift=projections[order_dotprods[160]],
                   mdl_index=-1, obj_index=objidx, extent=lm.extent,
                   contours=True, levels=7, delta=0.1, label=lm.obj_name)
    plot_scalebar(R=lm.maprad, length=max(int(lm.maprad/2), 1))
    plt.axis('off')
    plt.gcf().axes[0].get_xaxis().set_visible(False)
    plt.gcf().axes[0].get_yaxis().set_visible(False)
    plt.savefig("results/{}_kappa_pca{}_{}.pdf".format(lens, npca+1, 16), transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close()
    kappa_map_plot(lm, shift=projections[order_dotprods[840]],
                   mdl_index=-1, obj_index=objidx, extent=lm.extent,
                   contours=True, levels=7, delta=0.1, label=lm.obj_name)
    plot_scalebar(R=lm.maprad, length=max(int(lm.maprad/2), 1))
    plt.axis('off')
    plt.gcf().axes[0].get_xaxis().set_visible(False)
    plt.gcf().axes[0].get_yaxis().set_visible(False)
    plt.savefig("results/{}_kappa_pca{}_{}.pdf".format(lens, npca+1, 84), transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close()
